## Yêu cầu Đề bài: Dự án Phân tích Gian lận Giao dịch PaySim
Bối cảnh 🏢

Bạn là một nhà phân tích dữ liệu mới vào làm cho một công ty tài chính có tên là "PaySim". Công ty cung cấp dịch vụ chuyển tiền qua điện thoại di động. Gần đây, công ty nhận thấy các vụ lừa đảo đang gia tăng, gây thiệt hại về tiền bạc và làm giảm lòng tin của khách hàng.

Hệ thống hiện tại của công ty có một quy tắc rất đơn giản để cảnh báo (cột isFlaggedFraud), nhưng nó hoạt động không hiệu quả và bỏ lọt rất nhiều trường hợp.

Nhiệm vụ của bạn 📜

Giám đốc yêu cầu bạn phân tích bộ dữ liệu giao dịch trong một tháng để tìm ra những quy luật ẩn sau các hành vi gian lận.

Mục tiêu chính 🎯

Mục tiêu cuối cùng của bạn là tìm ra những "dấu hiệu" hoặc "dấu vân tay" đặc trưng của một giao dịch gian lận để từ đó xây dựng một hệ thống cảnh báo mới thông minh và hiệu quả hơn.

## Các Câu hỏi Điều tra chính (Lộ trình Phân tích)
Để đạt được mục tiêu trên, chúng ta sẽ lần lượt trả lời các câu hỏi sau. Mỗi câu hỏi là một bước trong cuộc điều tra của bạn:

Khám phá Tổng quan: Bức tranh toàn cảnh của dữ liệu trông thế nào? Có bao nhiêu giao dịch? Tỷ lệ gian lận là bao nhiêu? Dữ liệu có bị thiếu hay có gì bất thường không?

Phân tích Đặc điểm Gian lận: Hành vi gian lận có những đặc điểm gì nổi bật? Chúng thường là loại giao dịch nào (type)? Số tiền (amount) có khác biệt không?

Tìm "Dấu hiệu Vàng": Dấu hiệu nào là mạnh mẽ nhất để nhận biết một giao dịch là gian lận? (Ví dụ: Liệu có phải các giao dịch rút sạch tiền trong tài khoản đều là gian lận không?)

(Nâng cao) Xây dựng Mô hình: Dựa trên những dấu hiệu tìm được, làm thế nào để xây dựng một bộ quy tắc hoặc một mô hình đơn giản để "bắt" những giao dịch đáng ngờ này?

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


## Mũi điều tra 1: Khám phá Dữ liệu Tổng quan.

In [2]:
# Thêm dòng này vào một ô code ở đầu notebook và chạy nó một lần
pd.options.display.float_format = '{:,.2f}'.format
#Bức tranh toàn cảnh của dữ liệu trông thế nào?
#1 Kiểm tra kích thước dữ liệu
print("1. Kích thước Dữ liệu")
print(f"Bộ dữ liệu có {df.shape[0]} hàng và {df.shape[1]} cột.")
print('-' * 50)
#2 Kiểm tra ' Sức khỏe' của dữ liệu
# .info() sẽ cho ta biết kiểu dữ liệu của mỗi cột và liệu có dữ liệu nào bị thiếu không
df.info()
print("-" * 50)
# 3. Tóm tắt thống kê cho các cột số
# .describe() cho ta cái nhìn nhanh về các giá trị min, max, trung bình... của các cột số
print("\n## 3. Tóm tắt Thống kê cho các Cột số ##")
print(df.describe())
print("-" * 50)
# 4. Kiểm tra tỷ lệ gian lận( Quan trọng nhất)
# Như chúng ta đã biết cột isFraud cho chúng ta thông tin tỉ lệ gian lận
# .value_counts() đếm số lượng của mỗi giá trị trong cột 'isFraud'
# normalize=True sẽ chuyển số lượng đó thành phần trăm
print("\n## 4. Phân tích Tỷ Lệ Giao dịch Gian lận ##")
fraud_percentage = df['isFraud'].value_counts(normalize=True) * 100
print(fraud_percentage)

1. Kích thước Dữ liệu
Bộ dữ liệu có 6362620 hàng và 11 cột.
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB
--------------------------------------------------

## 3. Tóm tắt Thống kê cho các Cột số ##
              step        amount  oldbalanceOrg  newbalanceOrig  \
count 6,362,620.00  6,362,620.00   6,362,620.00    6,362,620.00   
mean        243.40    179,861.90     833,883.10      855,113.67   
std         142.33    603,858.23   2,888,

1. Có hơn 6 triệu giao dịch (con số lớn)
2. không có dữ liệu nào bị thiếu
3. Tỉ lệ gian lận là thấp (1000 giao dịch thì sẽ có 1 gian lận) nhưng điều chúng ta muốn là không có gian lận 

## Mũi điều tra 2: Phân tích Đặc điểm Gian lận

In [3]:
# --- Phân tích Đặc điểm Giao dịch Gian lận ---

# Tạo một DataFrame mới chỉ chứa các giao dịch gian lận để dễ phân tích
fraud_df = df[df['isFraud'] == 1]
# 1.Loại giao dịch (type) nào là gian lận?
print("## 1. Phân tích loại giao dịch của các vụ Gian Lận ##")
print("Các loại giao dịch gian lận bao gồm")
print(fraud_df['type'].value_counts())
print("-" * 50)
# 2. Số tiền(amount) trong các giao dịch gian lận có gì khác biệt ?
# Chúng ta sẽ so sánh thống kê của cột 'amount' giữa nhóm gian lận và nhóm bình thường
print("\n## 2. So sánh Số tiền giao dịch giữa nhóm Gian lận và Bình thường ##")
print("Thống kê số tiền cho các giao dịch BÌNH THƯỜNG:")
# Lấy 10000 mẫu ngẫu nhiên của nhóm bình thường để so sánh cho dễ nhìn
print(df[df['isFraud'] == 0]['amount'].describe())
print("\nThống kê số tiền cho các giao dịch GIAN LẬN:")
print(fraud_df['amount'].describe())
print("-" * 50)
# Một cách khác hiệu quả hơn để so sánh:
print("\n## So sánh hiệu quả hơn bằng groupby: ##")
comparison_table = df.groupby('isFraud')['amount'].describe()
comparison_table

## 1. Phân tích loại giao dịch của các vụ Gian Lận ##
Các loại giao dịch gian lận bao gồm
type
CASH_OUT    4116
TRANSFER    4097
Name: count, dtype: int64
--------------------------------------------------

## 2. So sánh Số tiền giao dịch giữa nhóm Gian lận và Bình thường ##
Thống kê số tiền cho các giao dịch BÌNH THƯỜNG:
count    6,354,407.00
mean       178,197.04
std        596,236.98
min              0.01
25%         13,368.40
50%         74,684.72
75%        208,364.76
max     92,445,516.64
Name: amount, dtype: float64

Thống kê số tiền cho các giao dịch GIAN LẬN:
count        8,213.00
mean     1,467,967.30
std      2,404,252.95
min              0.00
25%        127,091.33
50%        441,423.44
75%      1,517,771.48
max     10,000,000.00
Name: amount, dtype: float64
--------------------------------------------------

## So sánh hiệu quả hơn bằng groupby: ##


,count,mean,std,min,25%,50%,75%,max
isFraud,,,,,,,,
0,"6,354,407.00","178,197.04","596,236.98",0.01,"13,368.40","74,684.72","208,364.76","92,445,516.64"
1,"8,213.00","1,467,967.30","2,404,252.95",0.00,"127,091.33","441,423.44","1,517,771.48","10,000,000.00"


1. Số tiền trung bình (Mean) giao dịch gian lận cao gấp 8 lần so với những giao dịch bình thường
2. Giá trị 50% (Median) giao dịch 'điển hình' của gian lận cũng cao hơn gấp 6 lần so với giao dịch bình thường
3. Độ nhất quán (std - độ lệch chuẩn) Số tiền gian lận đa dạng và biến động hơn rất nhiều so với bình thường
4. Kết luận: Giao dịch gian lận thường nhắm đến các giao dịch giá trị cao, chúng không thực hiện giao dịch nhỏ vài nghìn đồng thấp nhất là 0 và 25% số giao dịch đã có giá trị trên 127k tức là còn cao hơn cả 50% của giao dịch bình thường. Chúng cố gắng lấy tiền rất đa dạng, không theo một quy luật cố định nào cả

##  Mũi điều tra 3: Tìm "Dấu hiệu Vàng".

Dựa trên những gì chúng ta đã khám phá, hãy cùng kiểm tra giả thuyết mới:

Giả thuyết: Kẻ gian thường cố gắng rút SẠCH tiền ra khỏi tài khoản trong một lần giao dịch.

Chúng ta có thể kiểm tra điều này trong dữ liệu bằng cách tìm các giao dịch thỏa mãn điều kiện: số dư mới của người gửi (newbalanceOrig) bằng 0.

In [4]:
# --- Tìm "Dấu hiệu Vàng" ---

# 1. Tạo một DataFrame mới chỉ chứa 2 loại giao dịch đáng ngờ
suspicious_types_df = df[df['type'].isin(['TRANSFER', 'CASH_OUT'])].copy()
# 2. Tìm các giao dịch mà tài khoản gốc bị rút sạch tiền
# Điều kiện: số dư mới của người gửi bằng 0, và số dư cũ lớn hơn 0( để chắc chắn rằng đây là một giao dịch rút tiền thực sự )
emptied_account_df = suspicious_types_df[
    (suspicious_types_df['newbalanceOrig'] == 0) & 
    (suspicious_types_df['oldbalanceOrg'] > 0)
]
# 3. Phân tích tỷ lệ gian lận chỉ trong nhóm đáng ngờ này
print("## Phân tích các giao dịch làm CẠN KIỆT tài khoản ##")
print(f"Tìm thấy {len(emptied_account_df)} giao dịch làm cạn kiệt tài khoản.")
print("\nTỷ lệ gian lận trong nhóm này là:")
fraud_rate_in_suspicious_group = emptied_account_df['isFraud'].value_counts(normalize=True) * 100
print(fraud_rate_in_suspicious_group)

## Phân tích các giao dịch làm CẠN KIỆT tài khoản ##
Tìm thấy 1188074 giao dịch làm cạn kiệt tài khoản.

Tỷ lệ gian lận trong nhóm này là:
isFraud
0   99.33
1    0.67
Name: proportion, dtype: float64


Đây là một manh mối tốt, một "dấu hiệu" có giá trị. Nó cho thấy hành vi rút sạch tiền có liên quan đến gian lận, nhưng nó chưa phải là "dấu hiệu vàng" mà chúng ta tìm kiếm. Tại sao? Bởi vì vẫn có đến 99.33% các giao dịch rút sạch tiền là giao dịch bình thường. Nếu chúng ta báo động tất cả các giao dịch này, chúng ta sẽ bắt nhầm rất nhiều người vô tội.

### Giả thuyết mới và mạnh mẽ hơn: "Dấu hiệu vàng" của một vụ lừa đảo là một giao dịch TRANSFER mà trong đó số tiền (amount) chuyển đi bằng chính xác số dư ban đầu (oldbalanceOrg).

In [5]:
# --- Tìm "Dấu hiệu Vàng" thực sự ---

# 1. Lọc ra các giao dịch TRANSFER mà số tiền chuyển đi bằng đúng số dư ban đầu
# Chúng ta cũng thêm điều kiện amount > 0 để loại bỏ các giao dịch 0 đồng
golden_signal_df = df[
    (df['type'] == 'TRANSFER') &
    (df['amount'] == df['oldbalanceOrg']) &
    (df['amount'] > 0)
]

print("## Phân tích các giao dịch TRANSFER rút toàn bộ số dư ##")
print(f"Tìm thấy {len(golden_signal_df)} giao dịch khớp với 'Dấu hiệu Vàng'.")

# 2. Phân tích tỷ lệ gian lận CHỈ trong nhóm siêu đáng ngờ này
print("\nTỷ lệ gian lận trong nhóm này là:")
final_fraud_rate = golden_signal_df['isFraud'].value_counts(normalize=True) * 100
print(final_fraud_rate)

## Phân tích các giao dịch TRANSFER rút toàn bộ số dư ##
Tìm thấy 3943 giao dịch khớp với 'Dấu hiệu Vàng'.

Tỷ lệ gian lận trong nhóm này là:
isFraud
1   100.00
Name: proportion, dtype: float64


## Phân tích Kết quả Quyết định
Tìm thấy: Có 3,943 giao dịch khớp với quy luật "chuyển khoản toàn bộ số dư".

Tỷ lệ gian lận: Trong số 3,943 giao dịch đó, 100% là gian lận.

Ý nghĩa:
Bạn đã tìm ra một quy luật có độ chính xác tuyệt đối. Nó không còn là "manh mối" hay "dấu hiệu" nữa, mà đã trở thành một quy tắc xác định (deterministic rule).

Điều này có nghĩa là, trong bộ dữ liệu này, bất kỳ khi nào có một giao dịch TRANSFER mà số tiền chuyển đi bằng chính xác số dư ban đầu, chúng ta có thể khẳng định chắc chắn 100% đó là một vụ lừa đảo.

## Phân tích Lỗ hổng
Tổng số vụ gian lận: 4116 (CASH_OUT) + 4097 (TRANSFER) = 8213 vụ.

Số vụ bắt được bởi "Dấu hiệu Vàng": 3943 vụ (đều là TRANSFER).

Kết luận của bạn: Chính xác. "Dấu hiệu Vàng" của chúng ta (amount == oldbalanceOrg cho giao dịch TRANSFER) rất mạnh, nhưng nó chỉ bắt được một loại tội phạm cụ thể, và chúng ta đã bỏ lọt:

Toàn bộ 4116 vụ gian lận qua CASH_OUT.

154 vụ gian lận qua TRANSFER (4097 - 3943 = 154) không tuân theo quy luật đó.

Điều này có nghĩa là có ít nhất hai "chủng" tội phạm khác nhau trong dữ liệu này, với những phương thức gây án khác nhau.

## Mũi điều tra Tiếp theo: Phân tích các Vụ án "Chưa được giải quyết"
Giờ chúng ta sẽ vào vai thám tử điều tra các vụ án còn lại. Liệu có một "Dấu hiệu Vàng" khác cho các giao dịch CASH_OUT không?

Giả thuyết: Có thể các vụ CASH_OUT gian lận cũng tuân theo quy luật tương tự: rút toàn bộ số tiền trong tài khoản.

In [6]:
# --- Điều tra các vụ án còn lại ---

# 1. Kiểm tra "Dấu hiệu Vàng" cho các giao dịch CASH_OUT
print("## 1. Kiểm tra các giao dịch CASH_OUT khớp với 'Dấu hiệu Vàng' ##")
cash_out_signal_df = df[
    (df['type'] == 'CASH_OUT') &
    (df['amount'] == df['oldbalanceOrg']) &
    (df['amount'] > 0)
]

# Phân tích tỷ lệ gian lận trong nhóm CASH_OUT này
if not cash_out_signal_df.empty:
    print(f"Tìm thấy {len(cash_out_signal_df)} giao dịch CASH_OUT khớp với quy luật.")
    print("Tỷ lệ gian lận trong nhóm này là:")
    print(cash_out_signal_df['isFraud'].value_counts(normalize=True) * 100)
else:
    print("Không tìm thấy giao dịch CASH_OUT nào khớp với quy luật 'amount == oldbalanceOrg'.")

print("-" * 30)

# 2. Xem xét 154 vụ TRANSFER còn lại
print("\n## 2. Phân tích 154 vụ TRANSFER gian lận chưa được giải quyết ##")

# Lọc ra các vụ TRANSFER gian lận
fraud_transfer_df = df[(df['type'] == 'TRANSFER') & (df['isFraud'] == 1)]

# Lọc ra những vụ KHÔNG khớp với "Dấu hiệu Vàng"
unsolved_transfers_df = fraud_transfer_df[fraud_transfer_df['amount'] != fraud_transfer_df['oldbalanceOrg']]

print(f"Số vụ TRANSFER chưa giải quyết: {len(unsolved_transfers_df)}")
print("Xem 5 vụ đầu tiên để tìm quy luật:")
print(unsolved_transfers_df.head())

## 1. Kiểm tra các giao dịch CASH_OUT khớp với 'Dấu hiệu Vàng' ##
Tìm thấy 4075 giao dịch CASH_OUT khớp với quy luật.
Tỷ lệ gian lận trong nhóm này là:
isFraud
1   100.00
Name: proportion, dtype: float64
------------------------------

## 2. Phân tích 154 vụ TRANSFER gian lận chưa được giải quyết ##
Số vụ TRANSFER chưa giải quyết: 154
Xem 5 vụ đầu tiên để tìm quy luật:
        step      type        amount     nameOrig  oldbalanceOrg  \
4440       4  TRANSFER 10,000,000.00     C7162498  12,930,418.44   
25875      8  TRANSFER  1,078,013.76  C1026280121           0.00   
60853      9  TRANSFER    994,453.20  C1121789613   1,437,370.87   
138559    11  TRANSFER  1,933,920.80  C1706582969           0.00   
217978    13  TRANSFER    123,194.95  C2143112877      79,466.45   

        newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
4440      2,930,418.44   C945327594            0.00            0.00        1   
25875             0.00   C277510102            0.00      97

## Phân tích Kết quả Cuối cùng
"Dấu hiệu Vàng" cho CASH_OUT:

Bạn đã tìm thấy một quy luật thứ hai với độ chính xác 100%. Bất kỳ giao dịch CASH_OUT nào có số tiền bằng số dư ban đầu đều là gian lận.

Quy luật này đã xác định được 4,075 vụ gian lận.

154 vụ TRANSFER còn lại:

Nhìn vào 5 ví dụ đầu tiên, chúng ta thấy một quy luật rất lạ. Ví dụ ở dòng 25875 và 138559, oldbalanceOrg (số dư ban đầu) là 0.00, nhưng vẫn có một giao dịch chuyển tiền lớn được thực hiện.

Ý nghĩa: Đây không phải là hành vi "rút sạch tiền" thông thường. Đây là một kiểu gian lận khác, tinh vi hơn, có thể là lợi dụng một lỗi hệ thống để tạo ra giao dịch từ một tài khoản không có tiền. Đây là một phát hiện quan trọng về một phương thức tấn công khác.

## Chúng ta đã bắt được bao nhiêu phần trăm tội phạm?
Bây giờ, hãy tổng kết hiệu quả của hai "Dấu hiệu Vàng" mà bạn đã tìm ra:

Tổng số vụ gian lận: 8,213

Số vụ TRANSFER bắt được bởi quy luật 1: 3,943

Số vụ CASH_OUT bắt được bởi quy luật 2: 4,075

Tổng số vụ bắt được: 3943 + 4075 = 8,018 vụ.

Hiệu quả: (8018 / 8213) * 100 ≈ 97.6%

Điều này thật đáng kinh ngạc. Chỉ với hai quy luật đơn giản, bạn đã xây dựng được một hệ thống có khả năng phát hiện hơn 97% tổng số các vụ lừa đảo!